In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# 크롬 드라이버 자동 업데이트
chrome_driver_path = ChromeDriverManager().install()

# Selenium 웹 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저를 띄우지 않고 백그라운드에서 실행
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)
real_estate = []
# 1페이지부터 5페이지까지 반복
for page_num in range(1, 6):
    # 1. https://omoney.kbstar.com/quics?page=C042015&cc=b037806:b037806에 진입한다.
    url = f"https://omoney.kbstar.com/quics?page=C042015&cc=b037806:b037806&page={page_num}"
    driver.get(url)
    
    # 2. html에서 class="tbl_list"안에서 href 목록을 구하는 코드
    tbl_list = driver.find_element(By.CLASS_NAME, "tbl_list")
    hrefs = [a.get_attribute("href") for a in tbl_list.find_elements(By.TAG_NAME, "a")]

    # 3. https://omoney.kbstar.com/에 href 목록들을 하나씩 붙여서 진입하며 id="view_cont"안에 있는 텍스트들을 크롤링
    for href in hrefs:
        driver.get(href)
        time.sleep(1)  # 페이지 로딩 대기
        view_cont = driver.find_element(By.ID, "view_cont").text
        #print(view_cont)
        #print("=" * 50)
        real_estate.append(view_cont)
        
print(len(real_estate))
driver.quit()

50


In [3]:
print(real_estate[48])

Q1. 낮은 점수로 청약 당첨이 가능하다고요?

청약제도 개편으로 가점이 낮은 세대의 국민주택규모 이하 주택 청약 당첨 기회가 확대돼요.

① 신규 아파트 공급 물량의 대부분을 차지하는 국민주택규모 이하 주택의 추첨 물량이 늘어나요. 
→ 가점이 높은 청약가입자의 당첨 기회는 상대적으로 감소해요.

 
② 일반 분양 예정인 청담 르엘, 반포 원펜타스, 잠실 래미안아이파크, 잠실 르엘 등 재건축 공사중인 아파트의 
국민주택규모 이하 아파트는 추첨제 적용으로 가점이 낮은 1주택자 당첨 기회가 확대돼요.

③ 1주택자의 갈아타기 수요가 꾸준한 국민주택규모 초과 주택의 추첨제 물량 감소로 큰 면적으로 이사 수요가 있는 1주택 실수요자의 당첨 기회는 상대적으로 감소해요.

 ★ 주택청약에 관심이 있다면 청약홈의 정보 및 부동산 청약 알림 앱을 설치하여 꾸준한 청약 정보 확인이 필요해요!

Q2. 2023 새로워진 청약제도, 주요 내용은?

① 전용면적 85㎡이하 주택 청약의 경우
기존에는 100% 가점제가 적용되었으나,
청약제도 개편으로 2023년 4월 이후 청약 주택의 경우
가점제 40%(전용60㎡이하), 70%(전용85㎡이하) 적용돼요.

② 국민주택규모(85㎡) 이하 주택의 추첨 물량을 늘리고
국민주택규모 초과 주택의 추첨 물량을 줄이는 방향으로 주택청약 제도가 새로워졌어요.

★ 지역 및 면적에 따라 선정 비율 등은 다릅니다. 자세한 내용은 PDF 첨부파일을 확인해 주세요.

[이창섭 KB국민은행 WM투자솔루션부 부동산전문위원]

더 많은 정보가 궁금하다면?

이번달 분양일정 확인하기

청년우대형 주택청약종합저축 총정리

주택청약종합저축 총정리

궁금한 부동산
